# Drug screening dataset

In [34]:
import gilda
import synapseclient
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
from rpy2 import robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

INFO: [2020-11-13 15:00:38] /Users/ben/Dropbox/postdoc/darpa/src/deft/adeft/recognize.py - OneShotRecognizer not available. Extension module for AlignmentBasedScorer is missing


In [59]:
syn = synapseclient.Synapse()
syn.login()

Welcome, bengyori!



INFO: [2020-11-13 15:08:07] synapseclient_default - Welcome, bengyori!



In [60]:
drug_data_path = syn.get("syn20684161").path
drug_data = pd.read_csv(drug_data_path, low_memory=False)
pd.DataFrame.head(drug_data)

,model_name,model_type,cellosaurus_id,organism_name,disease_name,disease_efo_id,symptom_name,symptom_efo_id,experiment_synapse_id,study_synapse_id,funder,drug_name,DT_explorer_internal_id,dosage_unit,drug_screen_id,dosage,response_type,response,response_unit
0,N10,cell line,NaN,human,no disease,NaN,no symptom,NaN,syn11373153.1,syn5610425,CTF,GPHR-00000018,159129,uM,1,"[0.0457246089549396,100]",AUC_Simpson,35.508784,NaN
1,N5,cell line,NaN,human,NF1,NaN,no symptom,NaN,syn11373157.1,syn5610425,CTF,GPHR-00000291,194936,uM,10,"[0.0457246089549396,100]",AUC_Simpson,4.018050,NaN
2,N5,cell line,NaN,human,NF1,NaN,no symptom,NaN,syn11373207.1,syn5610425,CTF,GPHR-00101658,313081,uM,100,"[0.0457246089549396,100]",AUC_Simpson,23.566992,NaN
3,N5,cell line,NaN,human,NF1,NaN,no symptom,NaN,syn11373765.1,syn5610425,CTF,GPHR-00224355,313317,uM,1000,"[0.0457246089549396,100]",AUC_Simpson,22.773713,NaN
4,Syn5,cell line,NaN,human,NF2,NaN,no symptom,NaN,syn12293953.1,syn2343195,CTF,NCGC00241342-02,150042,uM,10000,"[0.00078041474654,46.082949308]",AUC_Simpson,45.188508,NaN


In [61]:
pnf = ["ipNF05.5", "ipNF06.2A", "ipNF95.11b C/T", "ipnNF95.11C", "ipNF95.6", "ipNF05.5 (mixed clone)", "ipNF95.11b C"]

drug_data_pnf = (drug_data
      .query('response_type == "IC50_abs"')
      .query('model_name == @pnf')
      .groupby('drug_screen_id').filter(lambda x: len(x)==1)
      .assign(response = lambda x: [50 if a >= 50 else a for a in x['response']]))
drug_data_pnf

,model_name,model_type,cellosaurus_id,organism_name,disease_name,disease_efo_id,symptom_name,symptom_efo_id,experiment_synapse_id,study_synapse_id,funder,drug_name,DT_explorer_internal_id,dosage_unit,drug_screen_id,dosage,response_type,response,response_unit
80286,ipNF05.5 (mixed clone),cell line,NaN,human,NF1,NaN,pNF,658.0,syn5522644.1,syn4939906,NTAP,NCGC00013822-03,18372,uM,25926,"[0.000780415,46.08294931]",IC50_abs,6.076826e-12,uM
80287,ipNF05.5 (mixed clone),cell line,NaN,human,NF1,NaN,pNF,658.0,syn5522644.1,syn4939906,NTAP,NCGC00014352-03,313399,uM,25928,"[0.000780415,46.08294931]",IC50_abs,5.000000e+01,uM
80288,ipNF05.5 (mixed clone),cell line,NaN,human,NF1,NaN,pNF,658.0,syn5522644.1,syn4939906,NTAP,NCGC00014873-04,313422,uM,25931,"[0.000780415,46.08294931]",IC50_abs,5.000000e+01,uM
80289,ipNF05.5 (mixed clone),cell line,NaN,human,NF1,NaN,pNF,658.0,syn5522644.1,syn4939906,NTAP,NCGC00014891-03,313275,uM,25932,"[0.001560829,92.16589862]",IC50_abs,7.187728e-04,uM
80290,ipNF05.5 (mixed clone),cell line,NaN,human,NF1,NaN,pNF,658.0,syn5522644.1,syn4939906,NTAP,NCGC00015034-02,248772,uM,25938,"[0.000780415,46.08294931]",IC50_abs,1.458810e-04,uM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86838,ipNF95.11b C,cell line,NaN,human,NF1,NaN,pNF,658.0,syn8556314.3,syn4939906,NTAP,NCGC00347952-01,313821,uM,43365,"[7.8e-10,4.61e-05]",IC50_abs,6.682288e-07,uM
86839,ipNF95.11b C,cell line,NaN,human,NF1,NaN,pNF,658.0,syn8556314.3,syn4939906,NTAP,NCGC00347954-01,313752,uM,43367,"[7.8e-10,4.61e-05]",IC50_abs,1.515670e-05,uM
86840,ipNF95.11b C,cell line,NaN,human,NF1,NaN,pNF,658.0,syn8556314.3,syn4939906,NTAP,NCGC00347955-01,63637,uM,43368,"[7.8e-10,4.61e-05]",IC50_abs,7.404770e-05,uM
86841,ipNF95.11b C,cell line,NaN,human,NF1,NaN,pNF,658.0,syn8556314.3,syn4939906,NTAP,NCGC00347956-01,121999,uM,43369,"[7.8e-10,4.61e-05]",IC50_abs,6.483728e-05,uM


In [62]:
IC50_CUTOFF = 10
drug_data_pnf_positive = drug_data_pnf[drug_data_pnf.response < IC50_CUTOFF]

In [77]:
positive_drug_ids = sorted(list(drug_data_pnf_positive.DT_explorer_internal_id.unique()))
len(positive_drug_ids)

1072

## Target data

In [70]:
targetspath = syn.get('syn17091507')
readRDS = robjects.r['readRDS']
targets = readRDS(targetspath.path)

In [71]:
targets_filt = (targets
                .query('mean_pchembl > 6')
                .filter(["internal_id", "hugo_gene", "std_name"])
                .drop_duplicates())

In [73]:
targets.head()

,internal_id,hugo_gene,n_quantitative,mean_pchembl,cv,sd,IC50_nM,AC50_nM,EC50_nM,Potency_nM,Ki_nM,Kd_nM,n_qualitative,std_name,total_n,confidence,pchembl_d,pchembl_t,known_selectivity_index
1,3,HTR7,1,8.01,NaN,NaN,NaN,NaN,NaN,NaN,9.7,NaN,-2147483648,CHEMBL2413451,1,-0.229,8.01,10322.90,1.0
2,4,CHRNA4,1,7.54,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,-2147483648,CHEMBL204871,1,-0.229,15.08,4501.97,0.5
3,4,CHRNB2,1,7.54,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,-2147483648,CHEMBL204871,1,-0.229,15.08,4194.75,0.5
4,5,GSK3A,2,7.76,5.285076,0.410122,34.0,NaN,NaN,NaN,9.0,NaN,-2147483648,CHEMBL3582401,2,0.393,7.76,2034.63,1.0
5,6,FAAH,1,9.30,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,-2147483648,CHEMBL2386554,1,-0.229,9.30,6375.76,1.0


In [80]:
drug_names = sorted(targets.std_name[targets.internal_id.isin(set(positive_drug_ids))].unique())

In [82]:
len(drug_names)

661

In [108]:
from indra.ontology.bio import bio_ontology
drug_groundings = {}
for drug_name in drug_names:
    grounding = None
    # First handle names that are actually CHEMBL IDs
    if drug_name.startswith('CHEMBL'):
        grounding = ('CHEMBL', drug_name)
    # Next, try looking up some of these by standard name
    # This is needed because we have CHEMBL standard names in INDRA
    # that Gilda doesn't have e.g., TOZASERTIB
    else:
        for ns in ['CHEMBL', 'CHEBI']:
            id_from_standard_name = bio_ontology.get_id_from_name(ns, drug_name)
            if id_from_standard_name:
                grounding = (ns, id_from_standard_name)
                break
            else:
                id_from_standard_name = bio_ontology.get_id_from_name(ns, drug_name.upper())
                if id_from_standard_name:
                    grounding = (ns, id_from_standard_name)
                    break
        else:
            matches = gilda.ground(drug_name)
            if matches:
                grounding = (matches[0].term.db, matches[0].term.id)
    drug_groundings[drug_name] = grounding if grounding else None

In [109]:
'Ungrounded', len([k for k, v in drug_groundings.items() if not v])

('Ungrounded', 50)

In [110]:
for drug_name, grounding in drug_groundings.items():
    if not grounding:
        print(drug_name)

(3-{3-[[2-Chloro-3-(Trifluoromethyl)Benzyl](2,2-Diphenylethyl)Amino]Propoxy}Phenyl)Acetic Acid
(3e)-6'-Bromo-2,3'-Biindole-2',3(1h,1'h)-Dione 3-Oxime
2-({5-CHLORO-2-[(2-METHOXY-4-MORPHOLIN-4-YLPHENYL)AMINO]PYRIMIDIN-4-YL}AMINO)-N-METHYLBENZAMIDE
2-{[(1R,2S)-2-aminocyclohexyl]amino}-4-[(3-methylphenyl)amino]pyrimidine-5-carboxamide
3-[(3-(2-CARBOXYETHYL)-4-METHYLPYRROL-2-YL)METHYLENE]-2-INDOLINONE
4-(4-chlorobenzyl)-1-(7H-pyrrolo[2,3-d]pyrimidin-4-yl)piperidin-4-aminium
4-[(3S)-1-AZABICYCLO[2.2.2]OCT-3-YLAMINO]-3-(1H-BENZIMIDAZOL-2-YL)-6-CHLOROQUINOLIN-2(1H)-ONE
4-[3-(4-chlorophenyl)-2,1-benzisoxazol-5-yl]pyrimidin-2-amine
619C89
7-[4-(Dimethylamino)Phenyl]-N-Hydroxy-4,6-Dimethyl-7-Oxo-2,4-Heptadienamide
AM-630
AMG-458
AS1949490
AV-412
BIBR-1048
BMS-265246
BX-912
CGP-082996
CGP-60474
COMPOUND 10 [PMID: 19402633]
Cortexolone 17?-propionate
ERK INHIBITOR II
GNF-5837
GSK-J1
GW788388
Go-6893
IMD-0354
KRX-0601
KU-0060648
KU-60019
L-779450
METHYL N-({(2S,3S)-3-[(PROPYLAMINO)CARBONYL]OXIRAN-2-

In [113]:
from indra.statements import Agent, Inhibition
from indra.ontology.standardize import standardize_name_db_refs
drug_agents = []
for drug_name, grounding in drug_groundings.items():
    if not grounding:
        agent = Agent(drug_name)
        drug_agents.append(agent)
        continue
    standard_name, db_refs = standardize_name_db_refs({grounding[0]: grounding[1]})
    agent_name = standard_name if standard_name else drug_name
    agent = Agent(agent_name, db_refs=db_refs)
    drug_agents.append(agent)

In [115]:
match = gilda.ground('cell proliferation')[0]
standard_name, db_refs = standardize_name_db_refs({match.term.db: match.term.id})
cell_proliferation = Agent(standard_name, db_refs=db_refs)

In [118]:
from emmaa.model_tests import StatementCheckingTest
test_statements = [StatementCheckingTest(Inhibition(drug_agent, cell_proliferation))
                   for drug_agent in drug_agents]

INFO: [2020-11-13 15:55:01] covid_19.preprocess - Latest data release is 2020-11-12
INFO: [2020-11-13 15:55:01] pybel.config - no configuration found, using default sqlite connection sqlite:////Users/ben/.pybel/pybel_0.14.0_cache.db
INFO: [2020-11-13 15:55:15] indra_reading.readers.mti - Got MTI_EMAIL from environment
INFO: [2020-11-13 15:55:15] indra_reading.readers.mti - Got MTI_USERNAME from environment
INFO: [2020-11-13 15:55:15] indra_reading.readers.mti - Got MTI_PASSWORD from SSM
INFO: [2020-11-13 15:55:15] indra_reading.readers.mti - Got MTI_JARS_PATH from environment
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-

INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: 

INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: 

INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: 

INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: 

INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 15:55:16] emmaa.model_tests - Test configs: 

In [119]:
import pickle
with open('drug_screening_test_stmts.pkl', 'wb') as fh:
    pickle.dump(test_statements, fh)

cell population proliferation()

## Notebook for old dataset

In [11]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()


In [12]:
targetspath = syn.get('syn17091507')
readRDS = robjects.r['readRDS']

targets = readRDS(targetspath.path)

In [15]:

targets_filt = (targets
                .query('mean_pchembl > 6')
                .filter(["internal_id", "hugo_gene", "std_name"])
                .drop_duplicates())
    
pd.DataFrame.head(targets_filt)

,internal_id,hugo_gene,std_name
1,3,HTR7,CHEMBL2413451
2,4,CHRNA4,CHEMBL204871
3,4,CHRNB2,CHEMBL204871
4,5,GSK3A,CHEMBL3582401
5,6,FAAH,CHEMBL2386554


In [17]:
drug_data_path = syn.get("syn17462699").path
drug_data = pd.read_csv(drug_data_path, low_memory=False)
                       
pd.DataFrame.head(drug_data)

SynapseHTTPError: 403 Client Error: 
You do not have READ permission for the requested entity, syn17462699.